In [7]:
# Step 1: Libraries & Setup


# Data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Text preprocessing
import re
import string
import nltk
from nltk.corpus import stopwords

# Deep Learning
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D, Bidirectional
from tensorflow.keras.optimizers import Adam

# Model evaluation
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# HuggingFace for Transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from transformers import InputExample, InputFeatures

# Set seed for reproducibility
import random
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

# Step 2: Load & Structure Dataset

# Load the dataset (update path as needed)
olid_df = pd.read_excel("olid-training-v1.0.xlsx")

# Preview
print("Dataset Shape:", olid_df.shape)
olid_df.head()


Dataset Shape: (13240, 5)


,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you‚Äôre drunk!!! @USER #M...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [8]:

# Multi-Label Mapping

def map_toxicity(row):
    labels = {
        'toxic': 0,
        'obscene': 0,
        'threat': 0,
        'insult': 0,
        'identity hate': 0
    }

    if row['subtask_a'] == 'OFF':
        labels['toxic'] = 1

        if row['subtask_b'] == 'TIN':
            labels['insult'] = 1
            if row['subtask_c'] in ['IND', 'GRP']:
                labels['identity hate'] = 1
            elif row['subtask_c'] == 'OTH':
                labels['threat'] = 1

        elif row['subtask_b'] == 'UNT':
            labels['obscene'] = 1

    return pd.Series(labels)

# Apply the function
label_df = olid_df.apply(map_toxicity, axis=1)

# Merge with original
olid_df = pd.concat([olid_df, label_df], axis=1)

# Drop rows where all labels are 0 (optional but safe)
olid_df = olid_df[olid_df[['toxic', 'obscene', 'threat', 'insult', 'identity hate']].sum(axis=1) > 0]

# Check updated shape
print("Filtered Dataset Shape:", olid_df.shape)
olid_df.head()


Filtered Dataset Shape: (4400, 10)


,id,tweet,subtask_a,subtask_b,subtask_c,toxic,obscene,threat,insult,identity hate
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,1,1,0,0,0
1,90194,@USER @USER Go home you‚Äôre drunk!!! @USER #M...,OFF,TIN,IND,1,0,0,1,1
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,1,1,0,0,0
5,97670,@USER Liberals are all Kookoo !!!,OFF,TIN,OTH,1,0,1,1,0
6,77444,@USER @USER Oh noes! Tough shit.,OFF,UNT,NaN,1,1,0,0,0


In [9]:
# Step 3: Text Cleaning

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text)
    text = text.lower()                                  # Lowercase
    text = re.sub(r"@\w+", "", text)                     # Remove @mentions
    text = re.sub(r"http\S+|www\S+", "", text)           # Remove URLs
    text = re.sub(r"&[a-z]+;", "", text)                 # Remove HTML entities like &amp;
    text = re.sub(r"[^a-zA-Z\s]", "", text)              # Remove punctuation/special chars
    text = re.sub(r"\s+", " ", text).strip()             # Remove extra whitespace
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Apply to tweets
olid_df["clean_tweet"] = olid_df["tweet"].apply(clean_text)

# Preview cleaned text
olid_df[["tweet", "clean_tweet"]].head()


,tweet,clean_tweet
0,@USER She should ask a few native Americans wh...,ask native americans take
1,@USER @USER Go home you‚Äôre drunk!!! @USER #M...,go home youre drunk maga trump url
3,"@USER Someone should'veTaken"" this piece of sh...",someone shouldvetaken piece shit volcano
5,@USER Liberals are all Kookoo !!!,liberals kookoo
6,@USER @USER Oh noes! Tough shit.,oh noes tough shit


In [10]:
# Step 4: Tokenization & Padding


MAX_NUM_WORDS = 20000   # Max vocab size
MAX_SEQ_LEN = 100       # Max sequence length (tweets are short)

# Initialize tokenizer
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(olid_df["clean_tweet"])

# Convert to sequences
sequences = tokenizer.texts_to_sequences(olid_df["clean_tweet"])
word_index = tokenizer.word_index
print(f"Found {len(word_index)} unique tokens.")

# Pad sequences
X = pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post', truncating='post')

# Extract labels
y = olid_df[['toxic', 'obscene', 'threat', 'insult', 'identity hate']].values

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Found 10485 unique tokens.
Train shape: (3520, 100)
Test shape: (880, 100)


In [11]:
# Step 5: Load GloVe & Build Embedding Matrix


EMBEDDING_DIM = 100
embedding_index = {}

# Load glove.6B.100d.txt
with open("glove.6B.100d.txt", encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coeffs

print(f"Loaded {len(embedding_index)} word vectors from GloVe.")

# Build embedding matrix
embedding_matrix = np.zeros((MAX_NUM_WORDS, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i < MAX_NUM_WORDS:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

print("Embedding matrix shape:", embedding_matrix.shape)


Loaded 400000 word vectors from GloVe.
Embedding matrix shape: (20000, 100)


In [12]:
#  Step 6: BiLSTM Model


def create_bilstm_model():
    model = Sequential()

    # Embedding Layer (non-trainable)
    model.add(Embedding(input_dim=MAX_NUM_WORDS,
                        output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQ_LEN,
                        trainable=False))

    # Bidirectional LSTM
    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dropout(0.5))

    # Output layer: 5 units for multi-label
    model.add(Dense(5, activation='sigmoid'))  # Sigmoid for multi-label

    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=1e-3),
                  metrics=['accuracy'])

    return model

# Create and summarize the model
bilstm_model = create_bilstm_model()
bilstm_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2000000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              84480     
 l)                                                              
                                                                 
 dropout_38 (Dropout)        (None, 128)               0         
                                                                 
 dense (Dense)               (None, 5)                 645       
                                                                 
Total params: 2,085,125
Trainable params: 85,125
Non-trainable params: 2,000,000
_________________________________________________________________


In [13]:
#  Train the Model


history = bilstm_model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/5
88/88 [==============================] - 17s 140ms/step - loss: 0.3612 - accuracy: 0.7997 - val_loss: 0.3252 - val_accuracy: 1.0000
Epoch 2/5
88/88 [==============================] - 11s 125ms/step - loss: 0.3083 - accuracy: 1.0000 - val_loss: 0.3175 - val_accuracy: 1.0000
Epoch 3/5
88/88 [==============================] - 11s 120ms/step - loss: 0.2985 - accuracy: 1.0000 - val_loss: 0.3132 - val_accuracy: 1.0000
Epoch 4/5
88/88 [==============================] - 8s 93ms/step - loss: 0.2884 - accuracy: 0.9996 - val_loss: 0.3086 - val_accuracy: 1.0000
Epoch 5/5
88/88 [==============================] - 12s 138ms/step - loss: 0.2857 - accuracy: 1.0000 - val_loss: 0.3087 - val_accuracy: 1.0000


In [14]:
# Evaluate & Predict


# Predict
y_pred = bilstm_model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Report
print(classification_report(y_test, y_pred_binary, target_names=['toxic', 'obscene', 'threat', 'insult', 'identity hate']))


28/28 [==============================] - 3s 59ms/step
               precision    recall  f1-score   support

        toxic       1.00      1.00      1.00       880
      obscene       0.00      0.00      0.00       112
       threat       0.00      0.00      0.00        88
       insult       0.87      1.00      0.93       768
identity hate       0.77      1.00      0.87       680

    micro avg       0.88      0.92      0.90      2528
    macro avg       0.53      0.60      0.56      2528
 weighted avg       0.82      0.92      0.87      2528
  samples avg       0.88      0.90      0.89      2528



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
# Step 7: CNN Model


def create_cnn_model():
    model = Sequential()

    # Embedding Layer (non-trainable)
    model.add(Embedding(input_dim=MAX_NUM_WORDS,
                        output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQ_LEN,
                        trainable=False))

    # Convolution Layer
    model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
    model.add(GlobalMaxPooling1D())

    model.add(Dropout(0.5))

    # Output Layer
    model.add(Dense(5, activation='sigmoid'))

    # Compile
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=1e-3),
                  metrics=['accuracy'])

    return model

# Create and summarize
cnn_model = create_cnn_model()
cnn_model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          2000000   
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           64128     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout_39 (Dropout)        (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 2,064,773
Trainable params: 64,773
Non-trainable params: 2,000,000
_______________________________________

In [16]:
#  Train CNN Model


history_cnn = cnn_model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/5
88/88 [==============================] - 4s 34ms/step - loss: 0.3727 - accuracy: 0.8686 - val_loss: 0.3311 - val_accuracy: 1.0000
Epoch 2/5
88/88 [==============================] - 4s 48ms/step - loss: 0.3081 - accuracy: 0.9844 - val_loss: 0.3186 - val_accuracy: 1.0000
Epoch 3/5
88/88 [==============================] - 3s 29ms/step - loss: 0.2810 - accuracy: 0.9925 - val_loss: 0.3147 - val_accuracy: 1.0000
Epoch 4/5
88/88 [==============================] - 3s 29ms/step - loss: 0.2624 - accuracy: 0.9957 - val_loss: 0.3146 - val_accuracy: 1.0000
Epoch 5/5
88/88 [==============================] - 3s 31ms/step - loss: 0.2428 - accuracy: 0.9950 - val_loss: 0.3184 - val_accuracy: 1.0000


In [17]:
#  Evaluate CNN Model


y_pred_cnn = cnn_model.predict(X_test)
y_pred_cnn_binary = (y_pred_cnn > 0.5).astype(int)

print(classification_report(y_test, y_pred_cnn_binary, target_names=['toxic', 'obscene', 'threat', 'insult', 'identity hate']))


28/28 [==============================] - 0s 10ms/step
               precision    recall  f1-score   support

        toxic       1.00      1.00      1.00       880
      obscene       0.00      0.00      0.00       112
       threat       0.00      0.00      0.00        88
       insult       0.87      1.00      0.93       768
identity hate       0.77      0.99      0.87       680

    micro avg       0.88      0.92      0.90      2528
    macro avg       0.53      0.60      0.56      2528
 weighted avg       0.82      0.92      0.87      2528
  samples avg       0.88      0.90      0.89      2528



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
#  Step 8: GRU Model


def create_gru_model():
    model = Sequential()

    # Embedding Layer (non-trainable)
    model.add(Embedding(input_dim=MAX_NUM_WORDS,
                        output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQ_LEN,
                        trainable=False))

    # GRU Layer
    model.add(Bidirectional(tf.keras.layers.GRU(64, return_sequences=False)))
    model.add(Dropout(0.5))

    # Output
    model.add(Dense(5, activation='sigmoid'))

    # Compile
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=1e-3),
                  metrics=['accuracy'])

    return model

# Build and summarize
gru_model = create_gru_model()
gru_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          2000000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              63744     
 nal)                                                            
                                                                 
 dropout_40 (Dropout)        (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 5)                 645       
                                                                 
Total params: 2,064,389
Trainable params: 64,389
Non-trainable params: 2,000,000
_________________________________________________________________


In [19]:
# Train GRU Model


history_gru = gru_model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/5
88/88 [==============================] - 15s 107ms/step - loss: 0.3700 - accuracy: 0.8256 - val_loss: 0.3286 - val_accuracy: 1.0000
Epoch 2/5
88/88 [==============================] - 11s 129ms/step - loss: 0.3082 - accuracy: 1.0000 - val_loss: 0.3211 - val_accuracy: 1.0000
Epoch 3/5
88/88 [==============================] - 9s 106ms/step - loss: 0.2970 - accuracy: 1.0000 - val_loss: 0.3162 - val_accuracy: 1.0000
Epoch 4/5
88/88 [==============================] - 8s 93ms/step - loss: 0.2928 - accuracy: 1.0000 - val_loss: 0.3135 - val_accuracy: 1.0000
Epoch 5/5
88/88 [==============================] - 10s 113ms/step - loss: 0.2876 - accuracy: 1.0000 - val_loss: 0.3129 - val_accuracy: 1.0000


In [20]:
#  Evaluate GRU Model


y_pred_gru = gru_model.predict(X_test)
y_pred_gru_binary = (y_pred_gru > 0.5).astype(int)

print(classification_report(y_test, y_pred_gru_binary, target_names=['toxic', 'obscene', 'threat', 'insult', 'identity hate']))


28/28 [==============================] - 2s 29ms/step
               precision    recall  f1-score   support

        toxic       1.00      1.00      1.00       880
      obscene       0.00      0.00      0.00       112
       threat       0.00      0.00      0.00        88
       insult       0.87      1.00      0.93       768
identity hate       0.77      1.00      0.87       680

    micro avg       0.88      0.92      0.90      2528
    macro avg       0.53      0.60      0.56      2528
 weighted avg       0.82      0.92      0.87      2528
  samples avg       0.88      0.90      0.89      2528



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
!pip install transformers


In [8]:
#  Import Required Libraries

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer

# ==========================
# 📥 Load the OLID Excel File
# ==========================
olid_df = pd.read_excel("/content/olid-training-v1.0.xlsx")

# View column names to confirm
print("Columns:", olid_df.columns)

# ==========================
# 🧹 Clean the Tweets
# ==========================
def clean_text(text):
    text = re.sub(r"http\S+", "", str(text))      # Remove URLs
    text = re.sub(r"@\w+", "", text)              # Remove mentions
    text = re.sub(r"#\w+", "", text)              # Remove hashtags
    text = re.sub(r"[^a-zA-Z\s]", "", text)       # Remove special characters/numbers
    return text.lower().strip()

olid_df['clean_tweet'] = olid_df['tweet'].apply(clean_text)

# ==========================
#  Multi-Label Mapping
# ==========================

def map_toxicity(row):
    labels = {
        'toxic': 0,
        'obscene': 0,
        'threat': 0,
        'insult': 0,
        'identity hate': 0
    }

    if row['subtask_a'] == 'OFF':
        labels['toxic'] = 1

        if row['subtask_b'] == 'TIN':
            labels['insult'] = 1
            if row['subtask_c'] in ['IND', 'GRP']:
                labels['identity hate'] = 1
            elif row['subtask_c'] == 'OTH':
                labels['threat'] = 1

        elif row['subtask_b'] == 'UNT':
            labels['obscene'] = 1

    return pd.Series(labels)

# Apply the function
label_df = olid_df.apply(map_toxicity, axis=1)

# Merge with original
olid_df = pd.concat([olid_df, label_df], axis=1)

# Drop rows where all labels are 0 (optional but safe)
olid_df = olid_df[olid_df[['toxic', 'obscene', 'threat', 'insult', 'identity hate']].sum(axis=1) > 0]

# ==========================
#  Tokenize with DistilBERT
# ==========================
MAX_SEQ_LEN = 128

# Load tokenizer
distil_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def encode_distil(texts, tokenizer, max_len=MAX_SEQ_LEN):
    return tokenizer(
        list(texts),
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf',
        return_attention_mask=True
    )

# Encode the cleaned tweets
encoded_distil = encode_distil(olid_df["clean_tweet"], distil_tokenizer)

# Extract input tensors
X_ids_distil = encoded_distil['input_ids']
X_mask_distil = encoded_distil['attention_mask']

# ==========================
#  Define Multi-label Targets
# ==========================
# Update this if your Excel has different column names
label_columns = ['toxic', 'obscene', 'threat', 'insult', 'identity hate']
y_distil = olid_df[label_columns].values

# ==========================
#  Train-Test Split
# ==========================
# Convert EagerTensors to NumPy arrays before splitting
X_ids_distil_np = X_ids_distil.numpy()
X_mask_distil_np = X_mask_distil.numpy()

X_ids_distil_train, X_ids_distil_test, X_mask_distil_train, X_mask_distil_test, y_train_distil, y_test_distil = train_test_split(
    X_ids_distil_np, X_mask_distil_np, y_distil, test_size=0.2, random_state=42
)

Columns: Index(['id', 'tweet', 'subtask_a', 'subtask_b', 'subtask_c'], dtype='object')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
#  Step 10b: DistilBERT Model

from transformers import TFDistilBertForSequenceClassification # Import TFDistilBertForSequenceClassification
from tensorflow.keras.optimizers import Adam # Import Adam optimizer

distil_model = TFDistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=5,
    problem_type="multi_label_classification"
)

distil_model.compile(
    optimizer=Adam(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train the model
history_distil = distil_model.fit(
    [X_ids_distil_train, X_mask_distil_train],
    y_train_distil,
    epochs=3,
    batch_size=16,
    validation_split=0.2
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForS

Epoch 1/3
176/176 [==============================] - 2600s 15s/step - loss: 0.5132 - accuracy: 0.9233 - val_loss: 0.3207 - val_accuracy: 1.0000
Epoch 2/3
176/176 [==============================] - 2589s 15s/step - loss: 0.3255 - accuracy: 0.9972 - val_loss: 0.3265 - val_accuracy: 1.0000
Epoch 3/3
176/176 [==============================] - 2642s 15s/step - loss: 0.3538 - accuracy: 1.0000 - val_loss: 0.3128 - val_accuracy: 1.0000


In [15]:
import tensorflow as tf
from sklearn.metrics import classification_report

# Evaluate DistilBERT Model
y_pred_distil = distil_model.predict([X_ids_distil_test, X_mask_distil_test]).logits
y_pred_distil_binary = (tf.sigmoid(y_pred_distil).numpy() > 0.5).astype(int)

print(classification_report(y_test_distil, y_pred_distil_binary, target_names=['toxic', 'obscene', 'threat', 'insult', 'identity hate']))


28/28 [==============================] - 219s 8s/step
               precision    recall  f1-score   support

        toxic       1.00      1.00      1.00       880
      obscene       0.13      1.00      0.23       112
       threat       0.10      1.00      0.18        88
       insult       0.87      1.00      0.93       768
identity hate       0.77      1.00      0.87       680

    micro avg       0.57      1.00      0.73      2528
    macro avg       0.57      1.00      0.64      2528
 weighted avg       0.83      1.00      0.88      2528
  samples avg       0.57      1.00      0.73      2528

